In [23]:
# Importamos librerías

import pandas as pd
from __future__ import annotations
from random import random
from typing import List, Dict


In [24]:
partidos = pd.read_csv('../../data/partidos_limpio.csv')

In [26]:
class MarkovChainGraph:
    """
    Grafo para ejecutar el Algoritmo de Cadena de Markov
    """

    def __init__(self):
        self.transitions: Dict[str, Dict[str, float]] = {}

    def add_transition_probability(
        self, from_state: str, to_state: str, probability: float
    ) -> None:
        if from_state not in self.transitions:
            self.transitions[from_state] = {}
        self.transitions[from_state][to_state] = probability

    def get_states(self) -> List[str]:
        return list(self.transitions.keys())

    def transition(self, current_state: str) -> str:
        transition_probabilities = self.transitions.get(current_state, {})
        total_probability = sum(transition_probabilities.values())
        if total_probability == 0:
            return ""
        random_value = random()
        cumulative_probability = 0
        for next_state, probability in transition_probabilities.items():
            cumulative_probability += probability / total_probability
            if random_value <= cumulative_probability:
                return next_state
        return ""


def get_transition_matrix_from_dataframe(
    df: pd.DataFrame, home_col: str, away_col: str, result_col: str
) -> Dict[str, Dict[str, float]]:
    """
    Calcula la matriz de transición de una cadena de Markov basada en los resultados de los partidos de un DataFrame.

    Args:
    df (DataFrame): DataFrame que contiene los resultados de los partidos.
    home_col (str): Nombre de la columna para los equipos locales.
    away_col (str): Nombre de la columna para los equipos visitantes.
    result_col (str): Nombre de la columna para los resultados de los partidos.

    Returns:
    dict: Matriz de transición representada como un diccionario.
    """
    transitions = []

    # Convertir los resultados de los partidos en estados ('win', 'draw', 'lose')
    result_map = {'H': 'win', 'D': 'draw', 'A': 'lose'}
    df['from_state'] = df[result_col].map(result_map)

    # Calcular la siguiente transición como el estado siguiente después del partido
    df['to_state'] = df.groupby(home_col)['from_state'].shift(-1)

    # Eliminar filas con NaN resultantes de la última transición de cada equipo
    df.dropna(subset=['to_state'], inplace=True)

    # Contar ocurrencias de transición
    transition_counts = pd.crosstab(df['from_state'], df['to_state'])

    # Normalizar frecuencias para obtener probabilidades
    transition_probabilities = transition_counts.div(transition_counts.sum(axis=1), axis=0)

    # Convertir probabilidades de transición a un diccionario
    transition_matrix = transition_probabilities.to_dict(orient='index')

    return transition_matrix

def partidos_equipo(equipo, df):
    """
    Filtra los partidos en los que un equipo específico estuvo involucrado.

    Args:
    equipo (str): Nombre del equipo.
    df (DataFrame): DataFrame que contiene los datos de los partidos.

    Returns:
    DataFrame: DataFrame filtrado que contiene los partidos del equipo especificado.
    """
    return df[(df['Home'] == equipo) | (df['Away'] == equipo)].copy()

if __name__ == "__main__":
    # Ejemplo de DataFrame de partidos

    # Filtrar los partidos del Real Madrid
    partidos_real_madrid = partidos_equipo('Real Madrid', partidos)

    # Obtener la matriz de transición de los resultados de los partidos
    matriz_transicion = get_transition_matrix_from_dataframe(partidos_real_madrid, 'Home', 'Away', 'Results')

    print("Matriz de Transición:")
    for estado, transiciones in matriz_transicion.items():
        print(f"{estado}: {transiciones}")


Matriz de Transición:
draw: {'draw': 0.08333333333333333, 'lose': 0.25, 'win': 0.6666666666666666}
lose: {'draw': 0.15, 'lose': 0.4, 'win': 0.45}
win: {'draw': 0.21052631578947367, 'lose': 0.13157894736842105, 'win': 0.6578947368421053}
